<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/SG2_ADA_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

In [ ]:
%cd stylegan2-ada

In [ ]:
!gdown --id {GDrive ID} -O /content/network.pkl

In [ ]:
!python generate.py --help

## Generate Images

In [ ]:
!python generate.py generate-images --network="/content/ladiescrop.pkl" --seeds=0-10 --outdir="./out/"

## Trucation Traversal

In [ ]:
!python generate.py truncation-traversal --network="/content/ladiescrop.pkl" --seed=1 --start=-2.0 --stop=2.0 --increment=0.25 --outdir="./tt" --fps=30

## Interpolations

In [ ]:
!python generate.py generate-latent-walk --network="/content/ladiescrop.pkl" --walk-type="line-z" --seeds=0,2,5,0 --outdir="./z-walk"

In [ ]:
!python generate.py generate-latent-walk --network="/content/ladiescrop.pkl" --walk-type="line-w" --seeds=0,2,5,0 --outdir="./w-walk"

In [ ]:
!python generate.py generate-latent-walk --network="/content/ladiescrop.pkl" --walk-type="noiseloop" --start_seed=0 --outdir="./noise1"

## Near Neighbors

In [ ]:
!python generate.py generate-neighbors --network="/content/ladiescrop.pkl" --seeds=9 --outdir="./neighbors"

## Style Mixing

In [ ]:
!python style_mixing.py --outdir=out --rows=85,100,75,458,1500 --cols=55,821,1789,293 --network="/content/ladiescrop.pkl"

## Flesh Digressions

In [ ]:
!python aydao_flesh_digressions.py --pkl $pkl --psi=0.6 --radius_small=10.0 --radius_large=800.00

### Projection

In [ ]:
!python projector.py --outdir=out2 --target=/content/redbull.png --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

In [ ]:
!ffmpeg -i /content/stylegan2-ada/out2/proj.mp4 -filter:v "setpts=0.25*PTS" /content/stylegan2-ada/out2/out.mp4